## importing 

In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import requests
import os
import shutil
from shapely.geometry import Point, LineString, MultiPoint
import matplotlib
from ipywidgets import interact
import seaborn as sns
import math
import stats
import scipy
from dotenv import load_dotenv, find_dotenv
from urllib.parse import urljoin
import requests
from io import StringIO, BytesIO
import osmnx as ox
import time
import shapely
from folium import Map, CircleMarker, Vega, Popup, Marker, PolyLine, Icon, Choropleth, LayerControl
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
from folium.features import ColorLine, GeoJsonPopup
import json
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler

## testing remote opening of files

In [2]:
# constructor of google download links
dl_construct = 'https://drive.google.com/uc?export=download&id='
# id from share link google drive
file_id = '1zUzr93toblHkH2f3Zj5CEuD81qND7WYd'
csv = requests.get(f'{dl_construct}{file_id}').content
memory = BytesIO(csv)
df = pd.read_csv(memory)
df.shape

(8669, 110)

In [3]:
# list(df.columns)

In [4]:
dfd = df.drop(columns=['Link_ID','Unnamed: 0','Daily_Average_Traffic_Intensity','Average_Velocity_of_Vehicle_Traffic',
       'Median_of_velocity_of_Vehicle_Traffic',
       'FirstQuartil_of_velocity_of_Vehicle_Traffic',
       'ThirdQuartil_of_velocity_of_Vehicle_Traffic', 'Func_Class',
       'Speed_Cat','geometry', 'points'])

In [5]:
def scaling_numerical(data):
    numerical = data.columns.drop('linkid')
    scaler = RobustScaler()
    data_scaled = data.copy()
    for column in numerical:
        scaler.fit(data_scaled[[column]])
        data_scaled[column]=scaler.transform(data_scaled[[column]]) 
    return data_scaled
dfd = scaling_numerical(dfd)

In [6]:
dfd.head(10)

,linkid,feat_barrier_block,feat_barrier_bollard,feat_barrier_border_control,feat_barrier_bump_gate,feat_barrier_cattle_grid,feat_barrier_chain,feat_barrier_cycle_barrier,feat_barrier_debris,feat_barrier_door,...,feat_man_made_water_tap,feat_man_made_water_tower,feat_man_made_water_well,feat_man_made_water_works,feat_man_made_watermill,feat_man_made_windmill,feat_man_made_windpump,feat_man_made_works,feat_man_made_yes,sum_points
0,80216819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75
1,80216858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.00
2,80216859,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.75
3,80216883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.75
4,80216931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00
5,80217095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25
6,80217096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25
7,80217097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50
8,80217147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25
9,80217198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25


## New dataframe:

In [7]:
mine = pd.read_csv('wdl_data/preprocessed_data').drop(columns=['Unnamed: 0','geometry'])

In [8]:
final = mine.merge(dfd, on= 'linkid')

In [9]:
final_clean = final.drop(columns=['Func_Class','Speed_Cat','linkid','Average_Velocity_of_Vehicle_Traffic', 'Median_of_velocity_of_Vehicle_Traffic','FirstQuartil_of_velocity_of_Vehicle_Traffic',
                        'ThirdQuartil_of_velocity_of_Vehicle_Traffic', 'Max_speed', 'Speed_Diff_Median'])
final_clean.columns

Index(['Daily_Average_Traffic_Intensity', 'Speed_Diff_Mean', 'func_1',
       'func_2', 'func_3', 'speed_2', 'speed_3', 'speed_4', 'speed_5',
       'speed_6',
       ...
       'feat_man_made_water_tap', 'feat_man_made_water_tower',
       'feat_man_made_water_well', 'feat_man_made_water_works',
       'feat_man_made_watermill', 'feat_man_made_windmill',
       'feat_man_made_windpump', 'feat_man_made_works', 'feat_man_made_yes',
       'sum_points'],
      dtype='object', length=109)

## Starting with model

### baseline model 

The first thing after preprocessed all the model is to find a baseline score we can use to evaluate all the result we will get from now on. 
To do so sklearn provide a DummyRegressor method that basically will just calculate the mean and from it try to predict the target.

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression

# Prepare X and y
X = final_clean.drop(columns='Speed_Diff_Mean')
y = final['Speed_Diff_Mean']


X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3, 
                                                    random_state = 6)


baseline_model = DummyRegressor(strategy="mean") 
baseline_model.fit(X_train, y_train) 
baseline_model.score(X_test, y_test)

-0.0009663401601500698

- from the DummyRegressor we got a very low value in the test set. 
- we can now try to predict our test set with a more accurant evaluation method: the **Linear regression**.

In [11]:
model = LinearRegression().fit(X_train,y_train)
model.score(X_test,y_test)

0.4851844134672886

In [15]:
from sklearn.model_selection import cross_validate
# 5-Fold Cross validate model
model = LinearRegression()
cv_results = cross_validate(model, X, y, cv=5, 
                            scoring=['r2']
                           )
pd.DataFrame(cv_results) # Cross validation output

,fit_time,score_time,test_r2
0,0.079184,0.001701,-2.012143e+21
1,0.073108,0.001630,4.520015e-01
2,0.070804,0.001691,-5.395712e+20
3,0.076113,0.001599,-1.665741e+19
4,0.071861,0.001632,4.988699e-01


In [16]:
cv_results['test_r2'].mean()# Cross validation results

-5.136742799158137e+20

In [14]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression

linreg = LinearRegression().fit(X, y)
ridge = Ridge(alpha=0.2).fit(X, y)
lasso = Lasso(alpha=0.2).fit(X, y)

coefs = pd.DataFrame({
    "coef_linreg": pd.Series(linreg.coef_, index = X.columns),
    "coef_ridge": pd.Series(ridge.coef_, index = X.columns),
    "coef_lasso": pd.Series(lasso.coef_, index= X.columns)})\

coefs\
    .applymap(lambda x: int(x))\
    .style.applymap(lambda x: 'color: red' if x == 0 else 'color: black')

,coef_linreg,coef_ridge,coef_lasso
Daily_Average_Traffic_Intensity,0,0,0
func_1,-597750642371,0,0
func_2,-597750642371,0,0
func_3,-597750642371,0,0
speed_2,-468785005121,0,0
speed_3,-468785005121,0,0
speed_4,-468785005121,0,0
speed_5,-468785005121,0,0
speed_6,-468785005121,0,0
speed_7,-468785005121,0,0
